# Intro

In [1]:
import pandas as pd
import numpy as np
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
from sklearn.metrics.pairwise import cosine_similarity

#pip install gradio_client

In [2]:
#import csv filee interactions_train.csv
interactions = pd.read_csv('kaggle_data/interactions_train.csv')
items = pd.read_csv('kaggle_data/items.csv')

display(interactions.head())
display(items.head())



,u,i,t
0,4456,8581,1.687541e+09
1,142,1964,1.679585e+09
2,362,3705,1.706872e+09
3,1809,11317,1.673533e+09
4,4384,1323,1.681402e+09


,Title,Author,ISBN Valid,Publisher,Subjects,i
0,Classification décimale universelle : édition ...,NaN,9782871303336; 2871303339,Ed du CEFAL,Classification décimale universelle; Indexatio...,0
1,Les interactions dans l'enseignement des langu...,"Cicurel, Francine, 1947-",9782278058327; 2278058320,Didier,didactique--langue étrangère - enseignement; d...,1
2,Histoire de vie et recherche biographique : pe...,NaN,2343190194; 9782343190198,L'Harmattan,Histoires de vie en sociologie; Sciences socia...,2
3,Ce livre devrait me permettre de résoudre le c...,"Mazas, Sylvain, 1980-",9782365350020; 236535002X; 9782365350488; 2365...,Vraoum!,Moyen-Orient; Bandes dessinées autobiographiqu...,3
4,Les années glorieuses : roman /,"Lemaitre, Pierre, 1951-",9782702180815; 2702180817; 9782702183618; 2702...,Calmann-Lévy,France--1945-1975; Roman historique; Roman fra...,4


# 1. Collaborative Filtering


In [3]:
interactions = interactions.sort_values(["u", "t"])
interactions["pct_rank"] = interactions.groupby("u")["t"].rank(pct=True, method='dense')
interactions.reset_index(inplace=True, drop=True)

train_interactions = interactions[interactions["pct_rank"] < 0.8]
test_interactions = interactions[interactions["pct_rank"] >= 0.8]

print("Training set size:", train_interactions.shape[0])
print("Testing set size:", test_interactions.shape[0])

Training set size: 65419
Testing set size: 21628


In [4]:
# Create a user-item interaction matrix with binary values (1 if read, 0 otherwise)
binary_interaction_matrix = train_interactions.pivot_table(index='u', columns='i', values='t', aggfunc='count')
binary_interaction_matrix = binary_interaction_matrix.notnull().astype(int)

binary_interaction_matrix

i,0,1,2,3,4,5,6,7,8,9,...,15279,15280,15282,15283,15284,15285,15287,15288,15289,15290
u,,,,,,,,,,,,,,,,,,,,,
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7834,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7835,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# User To User

In [5]:
# Compute Cosine Similarity Between Users

user_similarity = cosine_similarity(binary_interaction_matrix)
user_similarity_df = pd.DataFrame(user_similarity, index=binary_interaction_matrix.index, columns=binary_interaction_matrix.index)
user_similarity_df

u,0,1,2,3,4,5,6,7,8,9,...,7828,7829,7830,7831,7832,7833,7834,7835,7836,7837
u,,,,,,,,,,,,,,,,,,,,,
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7833,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
7834,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
7835,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [6]:
k = 10

# 1. user-item scores for every user in one shot
scores = user_similarity_df.values @ binary_interaction_matrix.values        # (U, I)

# 2. indices of each user’s k best-scoring items (unsorted)
top_idx_unsorted = np.argpartition(-scores, k-1, axis=1)[:, :k]              # (U, k)

# 3. sort those k items per user so they’re really rank-ordered
rows   = np.arange(scores.shape[0])[:, None]                                  # (U, 1)
order  = np.argsort(-scores[rows, top_idx_unsorted], axis=1)
top_idx = top_idx_unsorted[rows, order]                                       # (U, k) sorted

# 4. look up the *labels* with NumPy → 2-D array → DataFrame
item_labels = binary_interaction_matrix.columns.to_numpy()                    # (I,)
top_labels  = item_labels[top_idx]                                            # (U, k)

recommendations = pd.DataFrame(
    top_labels,                              # the items
    index=binary_interaction_matrix.index,   # the users
    columns=range(k)                         # rank 0…9
)

# quick peek
print(recommendations.head())

     0    1    2    3    4    5    6    7    8     9
u                                                   
0   13    4   12   15   14   11    8   10    9     5
1   34   30   29   37   31   32   33   36   35  1573
2   46   58   53   49   56   82   64   75   45    67
3  149   40  138  155  128  142  143  156  133   139
4  202  198  191  203  193  201  197  196  199   195


In [7]:
def evaluate_precision_recall(recommendations, test_interactions, k=10):
    # Create a mapping from user to the set of items they interacted with in the test set
    test_user_items = test_interactions.groupby('u')['i'].apply(set)
    
    precisions = []
    recalls = []
    
    # Calculate precision and recall for each user with test interactions
    for user, rec_items in recommendations.iterrows():
        if user in test_user_items:
            relevant_items = test_user_items[user]
            rec_set = set(rec_items.values)
            true_positives = len(rec_set.intersection(relevant_items))
            precision = true_positives / len(rec_set)
            recall = true_positives / len(relevant_items) if len(relevant_items) > 0 else 0
            precisions.append(precision)
            recalls.append(recall)
            
    # Compute average precision and recall over all evaluated users
    avg_precision = sum(precisions) / len(precisions) if precisions else 0
    avg_recall = sum(recalls) / len(recalls) if recalls else 0
    
    print("Average Precision at {}: {}".format(k, avg_precision))
    print("Average Recall at {}: {}".format(k, avg_recall))
    
    return avg_precision, avg_recall

In [8]:
evaluate_precision_recall(recommendations, test_interactions, k=10)

Average Precision at 10: 0.05646848685889258
Average Recall at 10: 0.290439819786585


(0.05646848685889258, 0.290439819786585)

# Item to Item

In [9]:
item_similarity = cosine_similarity(binary_interaction_matrix.T)
item_similarity_df = pd.DataFrame(item_similarity, index=binary_interaction_matrix.columns, columns=binary_interaction_matrix.columns)
item_similarity_df

i,0,1,2,3,4,5,6,7,8,9,...,15279,15280,15282,15283,15284,15285,15287,15288,15289,15290
i,,,,,,,,,,,,,,,,,,,,,
0,1.000000,0.408248,0.333333,0.288675,0.109109,0.235702,0.288675,0.288675,0.218218,0.218218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.408248,1.000000,0.408248,0.353553,0.133631,0.288675,0.353553,0.353553,0.267261,0.267261,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.333333,0.408248,1.000000,0.288675,0.109109,0.235702,0.288675,0.288675,0.218218,0.218218,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.288675,0.353553,0.288675,1.000000,0.094491,0.204124,0.250000,0.250000,0.188982,0.188982,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.109109,0.133631,0.109109,0.094491,1.000000,0.077152,0.094491,0.094491,0.071429,0.142857,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15285,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
15287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
15288,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [10]:
k = 10

# 1. compute user-item scores using item similarity
# binary_interaction_matrix: (U, I)
# item_similarity_df: (I, I)
scores = binary_interaction_matrix.values @ item_similarity_df.values  # (U, I)

# 2. indices of each user's k best-scoring items (unsorted)
top_idx_unsorted = np.argpartition(-scores, k-1, axis=1)[:, :k]         # (U, k)

# 3. sort those k items per user to get rank-ordered recommendations
rows = np.arange(scores.shape[0])[:, None]                              # (U, 1)
order = np.argsort(-scores[rows, top_idx_unsorted], axis=1)
top_idx = top_idx_unsorted[rows, order]                                   # (U, k) sorted

# 4. look up the item labels (assuming column labels represent item ids)
item_labels = binary_interaction_matrix.columns.to_numpy()               # (I,)
top_labels = item_labels[top_idx]                                        # (U, k)

recommendations = pd.DataFrame(
    top_labels,                              # recommended items
    index=binary_interaction_matrix.index,   # users
    columns=range(k)                         # ranks 0…9
)

print(recommendations.head())

     0    1    2    3    4    5    6    7    8     9
u                                                   
0   18   11   14   15    1   12    0    2    6     3
1   33   36   31   35   32   29   30   37   34  2988
2   80   76   84   54   86   73   60   77   47    50
3  123  132  157  134  130  150  151  124  152   117
4  192  200  204  194  195  196  199  203  197   191


In [11]:
evaluate_precision_recall(recommendations, test_interactions)

Average Precision at 10: 0.05931360040826742
Average Recall at 10: 0.2816780249109841


(0.05931360040826742, 0.2816780249109841)

In [12]:
# Convert each row of recommendations to a space-separated string
recommendations_str = recommendations.apply(lambda row: ' '.join(row.astype(str)), axis=1)

# Export to CSV with a single-column header
recommendations_str.to_csv('recommendations.csv', index=True, header=['recommendation'])